In [2]:
import os, sys
sys.path.insert(0, os.environ['PROJECT_PATH'])

In [14]:
from __future__ import print_function
import pandas as pd
from gensim import corpora, utils
from gensim.parsing import preprocessing
from gensim.models.wrappers.dtmmodel import DtmModel

pd.options.display.max_columns = 999

from config.resources import path_to

In [4]:
import logging 

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [5]:
import matplotlib, matplotlib.pyplot as plt

%matplotlib inline
plt.style.use("ggplot")
matplotlib.rcParams.update({"font.size": 14})

DEBUG:matplotlib:CACHEDIR=/home/banerjee_abhimanyu3/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/banerjee_abhimanyu3/.cache/matplotlib/fontList.json
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [6]:
class DTMcorpus(corpora.textcorpus.TextCorpus):

    def get_texts(self):
        return self.input

    def __len__(self):
        return len(self.input)

In [7]:
jbv_df = pd.read_csv(path_to['jbv_meta'])

In [8]:
jbv_df.dropna(inplace=True, subset=[ 'AB' ])
jbv_df.sort_values('PY', inplace=True)
time_slices = jbv_df.PY.value_counts(ascending=True).tolist()

In [21]:
tokenized_abstracts = list(map(lambda a: utils.simple_preprocess(preprocessing.remove_stopwords(a)), jbv_df.AB))
dtm_path = os.path.join('..', 'data', 'models', 'dtm-linux64')

nb_topics = 22 
corpus = DTMcorpus(tokenized_abstracts)
path_to_model = path_to['dtm_model'].format('jbv', nb_topics)

if os.path.exists(path_to_model) and False:
    dtm_model = DtmModel.load(path_to_model)
else:
    dtm_model = DtmModel(dtm_path, corpus, time_slices, num_topics=nb_topics,
                     id2word=corpus.dictionary, initialize_lda=True)
    dtm_model.save(path_to_model)


INFO:gensim.corpora.textcorpus:Initializing dictionary
INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(9286 unique tokens: [u'writings', u'workaholism', u'four', u'ornati', u'localized']...) from 910 documents (total 124140 corpus positions)
INFO:gensim.models.wrappers.dtmmodel:serializing temporary corpus to /tmp/7d1189_train-mult.dat
INFO:gensim.corpora.bleicorpus:no word id mapping provided; initializing from corpus
INFO:gensim.corpora.bleicorpus:storing corpus in Blei's LDA-C format into /tmp/7d1189_train-mult.dat
DEBUG:smart_open.smart_open_lib:{'kw': {}, 'mode': 'wb', 'uri': '/tmp/7d1189_train-mult.dat'}
DEBUG:smart_open.smart_open_lib:encoding_wrapper: {'errors': 'strict', 'mode': 'wb', 'fileobj': <open file '/tmp/7d1189_train-mult.dat', mode 'wb' at 0x7fde50d61420>, 'encoding': None}
INFO:gensim.corpora.bleicorpus:saving vocabulary of 9286 words to /tmp/7d1189_train-mult.dat.vocab
DEBUG:smart_o

In [22]:
top_topic_descriptor_ls = dtm_model.show_topics(num_topics=-1, times=1)
f = open(path_to['dtm_descriptors'].format('jbv', nb_topics, '1'), 'w')
for _idx, topic_descr in enumerate(top_topic_descriptor_ls):
    print('topic ', _idx + 1, ': ', topic_descr)
    print('topic ', _idx + 1, ': ', topic_descr, file=f)
f.close()

topic  1 :  0.059*business + 0.016*the + 0.014*family + 0.013*founders + 0.013*managers + 0.011*planning + 0.010*owner + 0.010*management + 0.010*small + 0.010*factors
topic  2 :  0.076*entrepreneurs + 0.012*research + 0.011*individuals + 0.011*the + 0.010*study + 0.010*risk + 0.009*performance + 0.009*entrepreneurial + 0.008*satisfaction + 0.008*business
topic  3 :  0.063*technology + 0.031*firms + 0.023*external + 0.021*product + 0.020*firm + 0.015*strategies + 0.015*technological + 0.014*acquisition + 0.013*strategy + 0.013*the
topic  4 :  0.016*product + 0.014*success + 0.014*market + 0.013*new + 0.012*failure + 0.011*high + 0.010*the + 0.009*initial + 0.009*this + 0.009*related
topic  5 :  0.088*venture + 0.053*investors + 0.033*investment + 0.029*capital + 0.020*investments + 0.018*investor + 0.016*ventures + 0.014*capitalists + 0.013*stage + 0.012*market
topic  6 :  0.031*involvement + 0.031*family + 0.019*ownership + 0.017*ceos + 0.017*venture + 0.016*public + 0.015*this + 0.01

In [24]:
num_words, topic_idx = 10, 3
topics_over_time = [ [descr for _, descr in dtm_model.show_topic(topicid=topic_idx, time=time_idx, topn=num_words) ] \
                    for time_idx, _ in enumerate(time_slices)]
f = open(path_to['dtm_descr_evo'].format('jbv', topic_idx, nb_topics), 'w')
print('Topic descriptors evolution over {} time slices for topic {}\n'.format(len(time_slices), topic_idx))
print('Topic descriptors evolution over {} time slices for topic {}\n'.format(len(time_slices), topic_idx), file=f)
for descr_idx in range(num_words):
    descr_over_time = [ topics_over_time[time_idx][descr_idx] for time_idx, _ in enumerate(time_slices) ]
    print(" -> ".join(descr_over_time), '\n')
    print(" -> ".join(descr_over_time), '\n', file=f)
f.close()

Topic descriptors evolution over 27 time slices for topic 3

product -> product -> product -> product -> product -> product -> product -> market -> market -> market -> market -> market -> market -> market -> market -> market -> market -> market -> market -> market -> market -> market -> market -> market -> startups -> startups -> startups 

success -> success -> success -> success -> success -> success -> market -> success -> success -> success -> success -> success -> success -> success -> success -> success -> success -> success -> success -> success -> success -> success -> success -> success -> market -> market -> market 

market -> market -> market -> new -> new -> market -> success -> product -> product -> product -> product -> product -> product -> product -> product -> product -> product -> product -> product -> product -> startups -> startups -> startups -> startups -> success -> success -> success 

new -> new -> new -> market -> market -> new -> new -> new -> new -> new -> n